**MODELO CONCEITUAL (Versao 1.3)**

![title](../img/modelo-conceitual-1.3.jpg)

__MODELO RELACIONAL__

Doenca(__id__, nome, categoria)

Paciente(__id__, tipo, altura, peso, sexo, idade)

Sintoma(__codigo__, nome)

PacienteSintoma(__idPaciente__, __codSintoma__)<br/>
FK: idPaciente - Paciente(id)<br/>
FK: codSintoma - Sintoma(codigo)

PacienteHistoricoDoencas(__idPaciente__, __idDoenca__)<br/>
FK: idPaciente - Paciente(id)<br/>
FK: idDoenca - Doenca(id)

ContaMedico(__username__, senha, nome, sobrenome, email, numCRM, cidadeFederativa, situacao)

Remedio(__id__, principioAtivo, laboratorio, nomeVenda, apresentacao, classeTerapeutica, tipo, restricaoHospitalar, tarja)

Tratamento(__id__, __idRemedio__, apreDosagem, dosis_mcg, dosis_mg, dosis_ml, numGotas, repeticao, numVezesDia, numDias)<br/>
FK: idRemedio - Remedio(id)

Recomendacao(__codigo__, usernameMedico, idPaciente, __idTratamento__)<br/>
FK: usernameMedico - Medico(username)<br/>
FK: idPaciente - Paciente(id)

__SQL__

In [2]:
%defaultDatasource jdbc:h2:mem:db

In [3]:
DROP TABLE IF EXISTS Doenca;
DROP TABLE IF EXISTS Paciente;
DROP TABLE IF EXISTS Sintoma;
DROP TABLE IF EXISTS PacienteSintoma;
DROP TABLE IF EXISTS PacienteHistoricoDoencas;
DROP TABLE IF EXISTS ContaMedico;
DROP TABLE IF EXISTS Remedio;
DROP TABLE IF EXISTS Tratamento;
DROP TABLE IF EXISTS Recomendacao;

CREATE TABLE Doenca (
  id VARCHAR(5) NOT NULL,
  nome VARCHAR(50) NOT NULL,
  categoria VARCHAR(50) NULL,
  PRIMARY KEY(id)
) AS SELECT
    id,
    nome,
    categoria
FROM CSVREAD('../data/Doenca.csv');

CREATE TABLE Paciente (
  id VARCHAR(10) NOT NULL,
  tipo CHAR NOT NULL,
  altura DOUBLE NOT NULL,
  peso DOUBLE NOT NULL,
  sexo CHAR NOT NULL,
  idade INTEGER NOT NULL,
  PRIMARY KEY(id)
) AS SELECT
    id,
    tipo,
    altura,
    peso,
    sexo,
    idade
FROM CSVREAD('../data/Paciente.csv');

CREATE TABLE Sintoma (
  codigo VARCHAR(5) NOT NULL,
  nome VARCHAR(50) NOT NULL,
  PRIMARY KEY(codigo)
) AS SELECT
    codigo,
    nome
FROM CSVREAD('../data/Sintoma.csv');

CREATE TABLE PacienteSintoma (
  idPaciente VARCHAR(10) NOT NULL,
  codSintoma VARCHAR(5) NOT NULL,
  PRIMARY KEY(idPaciente, codSintoma),
  FOREIGN KEY(idPaciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codSintoma)
    REFERENCES Sintoma(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    idPaciente,
    codSintoma
FROM CSVREAD('../data/PacienteSintoma.csv');

CREATE TABLE PacienteHistoricoDoencas (
  idPaciente VARCHAR(10) NOT NULL,
  idDoenca VARCHAR(5) NOT NULL,
  PRIMARY KEY(idDoenca, idPaciente),
  FOREIGN KEY(idPaciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(idDoenca)
    REFERENCES Doenca(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    idPaciente,
    idDoenca
FROM CSVREAD('../data/PacienteHistoricoDoencas.csv');

CREATE TABLE ContaMedico (
  username VARCHAR(20) NOT NULL,
  senha VARCHAR(20) NOT NULL,
  nome VARCHAR(50) NOT NULL,
  sobrenome VARCHAR(50) NOT NULL,
  email VARCHAR(70) NOT NULL,
  numCRM INTEGER NOT NULL,
  cidadeFederativa VARCHAR(3) NOT NULL,
  situacao CHAR NOT NULL,
  PRIMARY KEY(username)
) AS SELECT
    username,
    senha,
    nome,
    sobrenome,
    email,
    numCRM,
    cidadeFederativa,
    situacao
FROM CSVREAD('../data/ContaMedico.csv');

CREATE TABLE Remedio (
  id VARCHAR(20) NOT NULL,
  principioAtivo VARCHAR(250) NOT NULL,
  laboratorio VARCHAR(150) NOT NULL,
  nomeVenda VARCHAR(150) NOT NULL,
  apresentacao VARCHAR(200) NULL,
  classeTerapeutica VARCHAR(150) NULL,
  tipo VARCHAR(150) NOT NULL,
  restricaoHospitalar VARCHAR(50) NULL,
  tarja VARCHAR(15) NULL,
  PRIMARY KEY(id)
) AS SELECT
    ID_REMEDIO,
    PRINCIPIO_ATIVO,
    LABORATORIO,
    PRODUTO,
    APRESENTACAO,
    CLASSE_TERAPEUTICA,
    TIPO_DE_PRODUTO,
    RESTRICAO_HOSPITALAR,
    TARJA
FROM CSVREAD('../data/Remedio.csv');

CREATE TABLE Tratamento (
  id VARCHAR(20) NOT NULL,
  idRemedio VARCHAR(20) NOT NULL,
  apreDosagem VARCHAR(20) NOT NULL,
  dosis_mcg DOUBLE NULL,
  dosis_mg DOUBLE NULL,
  dosis_ml DOUBLE NULL,
  numGotas DOUBLE NULL,
  repeticao INTEGER NULL,
  numVezesDia INTEGER NULL,
  numDias INTEGER NULL,
  PRIMARY KEY(id, idRemedio),
  FOREIGN KEY(idRemedio)
    REFERENCES Remedio(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    id,
    idRemedio,
    apreDosagem,
    dosis_mcg,
    dosis_mg,
    dosis_ml,
    numGotas,
    repeticao,
    numVezesDia,
    numDias
FROM CSVREAD('../data/Tratamento.csv');

CREATE TABLE Recomendacao (
  codigo VARCHAR(10) NOT NULL,
  usernameMedico VARCHAR(20) NOT NULL,
  idPaciente VARCHAR(10) NOT NULL,
  idTratamento VARCHAR(20) NOT NULL,
  PRIMARY KEY(codigo, idTratamento),
  FOREIGN KEY(usernameMedico)
    REFERENCES ContaMedico(username)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(idPaciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
    --fazendo referencia à tabela Tratamento se apresenta o erro: 'Primary key violation',
    --devido a que só se quere fazer referencia a um elemento (id) da superchave (id, idRemedio)
) AS SELECT
    codigo,
    usernameMedico,
    idPaciente,
    idTratamento
FROM CSVREAD('../data/Recomendacao.csv');

**CONSULTAS**

1) Quais sao os medicamentos escolhidos pelos doutores para tratar sintomas como: lacrimejo e coceira-olhos.<br/>
Obs: ter em conta que estos sintomas se podem apresentar juntos com outros.

In [ ]:
SELECT Re.nomeVenda
FROM Remedio AS Re
WHERE Re.id IN (SELECT Reme.id
                FROM Remedio AS Reme, Tratamento AS Tra, Recomendacao AS Reco, PacienteSintoma AS PaSin, Sintoma AS Sin
                WHERE Sin.nome = 'lacrimejo' OR Sin.nome = 'coceira-olhos'
                AND Sin.codigo = PaSin.codSintoma
                AND PaSin.idPaciente = Reco.idPaciente
                AND Reco.idTratamento = Tra.id
                AND Tra.idRemedio = Reme.id)

2) Que tipo de remedios sao receitados às pessoas que sofrem de pressao alta, quando apresentan congestionamento nasal.

In [5]:
SELECT DISTINCT nomeVenda, tarja
FROM Remedio
WHERE tarja = 'Tarja Preta' OR tarja = 'Tarja Vermelha'
ORDER BY nomeVenda

3) Quando os pacientes que sofrem de insuficiencia renal tem sintomas de alergia (lacrimejo, coceira-olhos, coriza ou vermelhidao), a quantidade de medicamento receitado segue algum padrao?

In [6]:
SELECT M.nome, R.id, R.nomeVenda
FROM Recomendacao Re, ContaMedico M, Tratamento T, Remedio R
WHERE Re.usernameMedico = M.username AND Re.idTratamento = T.id AND T.idRemedio = R.id AND
T.tempoUso > 5

4) Quando uma crian